In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# EDA

In [2]:
products = pd.read_csv(f"/home/adam/Documents/JobApplications/MarketDial/marketdial_ds_takehome/products_of_interest.csv", header=None)
stores = pd.read_csv(f"/home/adam/Documents/JobApplications/MarketDial/marketdial_ds_takehome/store_attributes.csv", usecols=list(range(1, 7)))
transactions = pd.read_csv(f"/home/adam/Documents/JobApplications/MarketDial/marketdial_ds_takehome/transactions.csv", usecols=list(range(1, 6)))

### Stores

In [3]:
stores.head()

,store_id,attribute_id,attribute_type,attribute_int_val,attribute_str_val,attribute_float_val
0,631,8,float,NaN,NaN,9447217.69
1,501,8,float,NaN,NaN,8807968.29
2,486,8,float,NaN,NaN,4038617.13
3,392,8,float,NaN,NaN,12222048.62
4,561,8,float,NaN,NaN,4298550.49


In [4]:
stores['store_id'].nunique()

337

In [5]:
stores['attribute_id'].nunique()

53

In [6]:
stores.groupby('attribute_type')['attribute_id'].nunique()

attribute_type
float      47
integer     2
string      4
Name: attribute_id, dtype: int64

In [7]:
string_cols = stores[stores['attribute_type'] == 'string']['attribute_id'].unique()

print("Strings: ", string_cols)

Strings:  [36 15  9 51]


In [8]:
# Check if the data is safe to pivot
    # i.e. no mislabeled attributes, missing values, etc

print(stores[stores['attribute_type'] == 'float'].count())
print()
print(stores[stores['attribute_type'] == 'integer'].count())
print()
print(stores[stores['attribute_type'] == 'string'].count())

store_id               14516
attribute_id           14516
attribute_type         14516
attribute_int_val          0
attribute_str_val          0
attribute_float_val    14516
dtype: int64

store_id               674
attribute_id           674
attribute_type         674
attribute_int_val      674
attribute_str_val        0
attribute_float_val      0
dtype: int64

store_id               1348
attribute_id           1348
attribute_type         1348
attribute_int_val         0
attribute_str_val      1348
attribute_float_val       0
dtype: int64


In [9]:
stores['value'] = \
    stores['attribute_int_val'] \
    .combine_first(stores['attribute_str_val']) \
    .combine_first(stores['attribute_float_val'])

In [10]:
# Check it worked as expected

# stores[stores['attribute_type'] == 'integer']
# 674+1348+14516

stores.count()

store_id               16538
attribute_id           16538
attribute_type         16538
attribute_int_val        674
attribute_str_val       1348
attribute_float_val    14516
value                  16538
dtype: int64

In [11]:
# Put stores into a more analysis-friendly format

stores = pd.pivot(stores, index='store_id', columns='attribute_id', values='value')

In [12]:
stores.head()

attribute_id,1,2,3,4,5,6,7,8,9,10,...,44,45,46,47,48,49,50,51,52,53
store_id,,,,,,,,,,,,,,,,,,,,,
338,4860.41,0.2087,0.9293,0.1495,NaN,6.0,0.1455,4787174.31,CO,4.3019,...,1.7362,78.8092,9.4235,0.0272,53186.0,0.3234,6.463,CO-AURORA-579,5499.0,NaN
339,2546.26,0.4454,0.4954,0.2152,0.0147,5.0,0.247,3868819.68,UT,2.0088,...,3.0488,63.6985,1.0423,0.046,22451.0,2.236,3.3932,UT-CEDAR CITY-244,12718.0,NaN
340,2446.6,1.13,1.0012,0.0001,NaN,22.0,0.1606,1624734.44,UT,2.5842,...,8.9465,NaN,3.091,0.1284,75574.0,1.2107,22.976,UT-SaltLake-539,137251.0,NaN
341,2526.21,0.17,0.2454,0.2416,NaN,4.0,0.1906,13185248.22,AZ,1.7727,...,2.2278,77.9783,0.4614,0.021,4118.0,3.3511,3.5039,AZ-SHOW LOW-388,3908.0,NaN
342,2368.64,0.0202,0.4319,0.2248,NaN,5.0,0.1988,7851917.14,OR,2.4074,...,2.4932,71.6176,0.7586,0.0473,9886.0,1.1633,3.7059,OR-BAKER CITY-439,5714.0,NaN


In [13]:
# Check data availability
attribute_pcts = stores.count() / len(stores)
attribute_pcts.sort_values()

# Drop attributes with <90% data populated (4 attributes)
print("Dropping Attributes: ", attribute_pcts[attribute_pcts < 0.9].index)
stores = stores.drop(attribute_pcts[attribute_pcts < 0.9].index, axis = 1)

Dropping Attributes:  Int64Index([5, 12, 39, 53], dtype='int64', name='attribute_id')


In [14]:
stores.head()

attribute_id,1,2,3,4,6,7,8,9,10,11,...,43,44,45,46,47,48,49,50,51,52
store_id,,,,,,,,,,,,,,,,,,,,,
338,4860.41,0.2087,0.9293,0.1495,6.0,0.1455,4787174.31,CO,4.3019,8.4082,...,-104.7386,1.7362,78.8092,9.4235,0.0272,53186.0,0.3234,6.463,CO-AURORA-579,5499.0
339,2546.26,0.4454,0.4954,0.2152,5.0,0.247,3868819.68,UT,2.0088,1.0779,...,-113.0741,3.0488,63.6985,1.0423,0.046,22451.0,2.236,3.3932,UT-CEDAR CITY-244,12718.0
340,2446.6,1.13,1.0012,0.0001,22.0,0.1606,1624734.44,UT,2.5842,5.6805,...,-111.887,8.9465,NaN,3.091,0.1284,75574.0,1.2107,22.976,UT-SaltLake-539,137251.0
341,2526.21,0.17,0.2454,0.2416,4.0,0.1906,13185248.22,AZ,1.7727,1.5542,...,-110.0182,2.2278,77.9783,0.4614,0.021,4118.0,3.3511,3.5039,AZ-SHOW LOW-388,3908.0
342,2368.64,0.0202,0.4319,0.2248,5.0,0.1988,7851917.14,OR,2.4074,0.8598,...,-117.8227,2.4932,71.6176,0.7586,0.0473,9886.0,1.1633,3.7059,OR-BAKER CITY-439,5714.0


##### Impute Missing Data

In [66]:
# There is no missing data for the string columns...thank you

numeric_cols = [col for col in stores.columns if col not in string_cols]

stores[numeric_cols] = stores[numeric_cols].apply(pd.to_numeric)
stores[numeric_cols] = stores[numeric_cols].fillna(stores[numeric_cols].median())

In [86]:
# Attribute 51 is unique for each store - dropped to allow for dummy variables

stores[string_cols[3]].nunique()

stores.drop(columns = string_cols[3], inplace = True)
string_cols = np.delete(string_cols, 3)

In [90]:
# Convert strings to categorical


array(['CO', 'UT', 'AZ', 'OR', 'WY', 'ID', 'WA', 'SD', 'NV', 'NE', 'MT'],
      dtype=object)